In [134]:
from datetime import datetime as dt
from IPython.display import IFrame

import os 
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [26]:
API_URL = 'https://hypothes.is/api/'
API_REF_URL = 'https://h.readthedocs.io/en/latest/api-reference/'
API_TOKEN = '6879-zbVCq3wU5_xddQb4BR85liZRfhL4Iqz6CmOYN56o2Eo'

Todo
- [x] [get api methods and urls](#API-Methods-and-URLs)
- [x] [create new group (Token Whitepapers)](#Create-Group)
- [x] [get group](#Get-Group)
- [x] [search annotations](#Search)
 - [x] by group
 - [ ] by keyword
 - [ ] by time

- [ ] [create a visual of annotations over time](#Visualize-Annotations)

### API Methods and URLs

In [48]:
res = requests.get(API_URL)
api = res.json()['links']

In [49]:
api.keys()

dict_keys(['profile', 'search', 'group', 'links', 'user', 'groups', 'annotation'])

### Create Group

In [44]:
IFrame(f'{API_REF_URL}#operation/createGroup', 600, 500)

In [50]:
api['group']['create']

{'url': 'https://hypothes.is/api/groups',
 'method': 'POST',
 'desc': 'Create a new group'}

In [68]:
new_group_name = 'Token Whitepapers'
new_group_desc = 'Annotation group for token whitepapers'

In [69]:
res = requests.request(
    api['group']['create']['method'],
    api['group']['create']['url'],
    headers={
        'Authorization': f'Bearer {API_TOKEN}',
        'Accept': 'application/json'
    },
    json={
        'name': new_group_name,
        'description': new_group_desc
    }
)

res.text

'{"scopes": {"uri_patterns": [], "enforced": false}, "name": "Token Whitepapers", "links": {"html": "https://hypothes.is/groups/LALjzxP7/token-whitepapers"}, "id": "LALjzxP7", "groupid": null, "scoped": false, "organization": null, "type": "private", "public": false}'

### Get Group

In [88]:
IFrame(f'{API_REF_URL}#op', 600, 500)

In [89]:
api['groups']

{'read': {'url': 'https://hypothes.is/api/groups',
  'method': 'GET',
  'desc': "Fetch the user's groups"}}

In [90]:
res = requests.request(
    api['groups']['read']['method'],
    api['groups']['read']['url'],
    headers={
        'Authorization': f'Bearer {API_TOKEN}',
        'Accept': 'application/json'
    }
)

groups = res.json()

In [91]:
len(groups)

10

In [92]:
groups[0].keys()

dict_keys(['name', 'links', 'id', 'groupid', 'scoped', 'organization', 'type', 'public'])

In [94]:
new_group = list(filter(lambda g: g['name']==new_group_name, groups))[0]
new_group

{'name': 'Token Whitepapers',
 'links': {'html': 'https://hypothes.is/groups/LALjzxP7/token-whitepapers'},
 'id': 'LALjzxP7',
 'groupid': None,
 'scoped': False,
 'organization': None,
 'type': 'private',
 'public': False}

In [95]:
api['group']['read']

{'url': 'https://hypothes.is/api/groups/:id',
 'method': 'GET',
 'desc': 'Fetch a group'}

In [96]:
res = requests.request(
    api['group']['read']['method'],
    api['group']['read']['url'].replace(':id',new_group['id']),
    headers={
        'Authorization': f'Bearer {API_TOKEN}',
        'Accept': 'application/json'
    }
)

new_group['data'] = res.json()

In [97]:
new_group['data']

{'name': 'Token Whitepapers',
 'links': {'html': 'https://hypothes.is/groups/LALjzxP7/token-whitepapers'},
 'id': 'LALjzxP7',
 'groupid': None,
 'scoped': False,
 'organization': None,
 'type': 'private',
 'public': False}

In [98]:
new_group['id']

'LALjzxP7'

### Search

In [100]:
IFrame(f'{API_REF_URL}#operation/search', 600, 500)

In [102]:
api['search']

{'url': 'https://hypothes.is/api/search',
 'method': 'GET',
 'desc': 'Search for annotations'}

In [156]:
res = requests.request(
    api['search']['method'],
    api['search']['url'],
    headers={
        'Authorization': f'Bearer {API_TOKEN}',
        'Accept': 'application/json'
    },
    params={
        'group': new_group['id'],
        'user:': 'malcolmjmr',
        'limit': 200
    }
)

query_result = res.json()

In [157]:
query_result['total']

27

In [158]:
query_result['rows'][0]

{'updated': '2019-02-05T18:24:49.608813+00:00',
 'group': 'LALjzxP7',
 'target': [{'source': 'https://bitcoin.org/bitcoin.pdf',
   'selector': [{'type': 'TextPositionSelector', 'end': 1280, 'start': 1275},
    {'exact': 'proof',
     'prefix': '  longestproof-of-work chain as ',
     'type': 'TextQuoteSelector',
     'suffix': ' of what happened while they wer'}]}],
 'links': {'json': 'https://hypothes.is/api/annotations/UrMJWilzEemJkTc4McKOYw',
  'html': 'https://hypothes.is/a/UrMJWilzEemJkTc4McKOYw',
  'incontext': 'https://hyp.is/UrMJWilzEemJkTc4McKOYw/bitcoin.org/bitcoin.pdf'},
 'tags': ['requirement'],
 'text': '',
 'created': '2019-02-05T18:24:49.608813+00:00',
 'uri': 'https://bitcoin.org/bitcoin.pdf',
 'flagged': False,
 'user_info': {'display_name': None},
 'moderation': {'flagCount': 0},
 'user': 'acct:malcolmjmr@hypothes.is',
 'hidden': False,
 'document': {'title': ['bitcoin.pdf']},
 'id': 'UrMJWilzEemJkTc4McKOYw',
 'permissions': {'read': ['group:LALjzxP7'],
  'admin': ['a

In [161]:
annotations = json_normalize(query_result['rows'])
for col in ['created','updated']:
    annotations[col] = pd.to_datetime(annotations[col])
    
annotations['concept'] = annotations.target.apply(lambda t: t[0]['selector'][1]['exact'].strip().lower())
annotations['tags'] = annotations.tags.apply(lambda t: ','.join(t))
annotations

annotations.dtypes

created                   datetime64[ns]
document.title                    object
flagged                             bool
group                             object
hidden                              bool
id                                object
links.html                        object
links.incontext                   object
links.json                        object
moderation.flagCount               int64
permissions.admin                 object
permissions.delete                object
permissions.read                  object
permissions.update                object
tags                              object
target                            object
text                              object
updated                   datetime64[ns]
uri                               object
user                              object
user_info.display_name            object
concept                           object
dtype: object

In [172]:
annotations[['concept','tags','links.incontext','uri']].set_index('concept')

,tags,links.incontext,uri
concept,,,
proof,requirement,https://hyp.is/UrMJWilzEemJkTc4McKOYw/bitcoin....,https://bitcoin.org/bitcoin.pdf
rejoin,requirement,https://hyp.is/RpxYdClzEem8GcsPtIBiDQ/bitcoin....,https://bitcoin.org/bitcoin.pdf
leave,requirement,https://hyp.is/P5zTqilzEemeIdsZZuqFSA/bitcoin....,https://bitcoin.org/bitcoin.pdf
message,"requirement,solution",https://hyp.is/JMKhQClzEemkhq_VlIQedw/bitcoin....,https://bitcoin.org/bitcoin.pdf
broadcast,solution,https://hyp.is/6swf7CluEemoEzdheYbNCA/bitcoin....,https://bitcoin.org/bitcoin.pdf
network,"solution,requirement",https://hyp.is/eE10CClyEemqxc-OkRxPJA/bitcoin....,https://bitcoin.org/bitcoin.pdf
attack,problem,https://hyp.is/3jet5iluEeml-YdeUcmiIw/bitcoin....,https://bitcoin.org/bitcoin.pdf
node,"solution,requirement",https://hyp.is/VSNUoilyEemgqIPxV4YmZw/bitcoin....,https://bitcoin.org/bitcoin.pdf
majority,"solution,problem",https://hyp.is/1yvAoCluEemOXrOGRlaDcA/bitcoin....,https://bitcoin.org/bitcoin.pdf


In [163]:
annotations.groupby('tags').concept.count()

tags
problem                  3
requirement              6
requirement,problem      1
requirement,solution     1
solution                12
solution,problem         2
solution,requirement     2
Name: concept, dtype: int64

In [164]:
annotations[annotations.tags.str.contains('requirement')].concept

0                 proof
1                rejoin
2                 leave
3               message
5               network
7                  node
17          transaction
21                trust
24       onlinepayments
25    electronic   cash
Name: concept, dtype: object

Trusted online payments with electronic cash. Nodes can message transaction to the network provide proof to other nodes whether they leave and rejoin (have a dedicated connection) to the network   

In [165]:
annotations[annotations.tags.str.contains('problem')].concept

6                    attack
8                  majority
17              transaction
19             double-spend
20              third party
23    financial institution
Name: concept, dtype: object

All without the network getting attacked through double spend or majority. And without the need for third party financial instutions. 

In [166]:
annotations[annotations.tags.str.contains('solution')].concept

3                message
4              broadcast
5                network
7                   node
8               majority
9              cpu power
10                   cpu
11                  pool
12         longest chain
13                record
14         proof-of-work
15                 chain
16                  hash
18             timestamp
20           third party
22    digital signatures
26          peer-to-peer
Name: concept, dtype: object

Node(s) broadcast message(s) with digital signiatures to peer to peer network. Node(s) create hash as proof of work (dedicated cpu power). Those with majority of pooled cpu power determine the record. They timestamp and hash (unique identifier) of message that is linked to chain hash. Longest chain is the one nodes trust. 


In [ ]:
text = textract.process(